In [ ]:
from bspump.http.web.server import *
from bspump.jupyter import *
import asyncio

In [ ]:
new_pipeline("WebServerPipeline")

In [1]:
print("Registering source")


@register_source
def source(app, pipeline):
    return WebFormSource(
        app,
        pipeline,
        route="/",
        fields=[
            TextField("user_name", display="User Name"),
            TextField("user_email", display="Email"),
            IntField("user_age", display="Age"),
            CheckboxField("user_verified", display="Verified"),
            CheckboxField("newsletter", display="Newsletter Subscription"),
            CheckboxField("notifications_email", display="Email Notifications"),
            CheckboxField("notifications_sms", display="SMS Notifications"),
            IntField("order_id", display="Order ID"),
            TextField("language", display="Language"),
            TextField("keywords", display="Keywords (comma-separated)", multiline=True),
        ],
    )

Registering source


NameError: name 'register_source' is not defined

In [ ]:
@async_step
async def output(inject, ctx):
    print("Processing form data...")
    print(ctx["form"])
    await asyncio.sleep(1)

    try:
        form_data = ctx["form"]

        ctx["response"] = {
            "user": {
                "name": form_data["user_name"],
                "email": form_data["user_email"],
                "profile": {
                    "age": form_data["user_age"],
                    "verified": form_data["user_verified"],
                    "preferences": {
                        "newsletter": form_data["newsletter"],
                        "notifications": {
                            "email": form_data["notifications_email"],
                            "sms": form_data["notifications_sms"],
                        },
                    },
                },
            },
            "order": {
                "id": form_data["order_id"],
                "shipped": False,
            },
            "settings": {
                "language": form_data["language"],
                "keywords": form_data["keywords"].split(","),
            },
        }

    except KeyError as e:
        ctx["response"] = {"error": f"Missing field: {str(e)}"}
    except Exception as e:
        ctx["response"] = {"error": f"Unexpected error: {str(e)}"}

    await inject(ctx)

In [ ]:
@register_sink
def sink(app, pipeline):
    return JSONWebSink(app, pipeline)

In [ ]:
end_pipeline()